In [ ]:
from dotenv import load_dotenv  # Load biến môi trường từ file .env
load_dotenv() 
import os 
import requests 
import pandas as pd 
import time

# URL gốc để lấy dữ liệu xếp hạng người chơi từ Riot API
root = 'https://na1.api.riotgames.com/lol/league/v4/'

# Gửi yêu cầu để lấy dữ liệu người chơi trong 3 bậc rank cao nhất: Challenger, Grandmaster, Master
chall = requests.get(root+'challengerleagues/by-queue/RANKED_SOLO_5x5?api_key='+riot_api_key)
gm  = requests.get(root+'grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key='+riot_api_key)
m1  = requests.get(root+'masterleagues/by-queue/RANKED_SOLO_5x5?api_key='+riot_api_key)

# Chuyển dữ liệu thành DataFrame để xử lý
# 'entries' chứa danh sách người chơi trong từng bậc rank

df_chall = pd.DataFrame(chall.json()['entries'])
df_gm = pd.DataFrame(gm.json()['entries'])
df_m1 = pd.DataFrame(m1.json()['entries'])

# Sắp xếp người chơi theo điểm rank từ cao đến thấp
df_chall_sorted = df_chall.sort_values(by="leaguePoints", ascending=False)
df_gm_sorted = df_gm.sort_values(by="leaguePoints", ascending=False)
df_m1_sorted = df_m1.sort_values(by="leaguePoints", ascending=False)

# Gộp danh sách người chơi từ 3 bậc rank
# Chỉ lấy cột 'puuid' để sử dụng trong các truy vấn tiếp theo
df_all = pd.concat([df_chall_sorted, df_gm_sorted, df_m1_sorted], ignore_index=True)['puuid']

# Chia danh sách người chơi thành 2 phần:
df_top600 = df_all.iloc[:600]    # 600 người chơi đứng đầu
df_remaining = df_all.iloc[600:1600]  # 1000 người chơi tiếp theo

# Hàm lấy danh sách ID trận đấu từ puuid của người chơi
def get_id_match(puuid = None): 
    root_url = 'https://americas.api.riotgames.com'
    endpoint = f'/lol/match/v5/matches/by-puuid/{puuid}/ids'
    response = requests.get(root_url+endpoint+'?start=0&count=1&api_key='+riot_api_key)
    return response.json()

# Hàm lấy thông tin người chơi từ ID trận đấu
def get_player_from_match(matchId=None):
    root_url = 'https://americas.api.riotgames.com'
    endpoint = f'/lol/match/v5/matches/{matchId}'
    response = requests.get(root_url + endpoint + '?api_key=' + riot_api_key)
    x = response.json()['info']['participants'] 
    for player in x: 
        if player['teamPosition'] == 'MIDDLE':  # Lọc người chơi đi đường giữa (MID)
            return player

# --- Xử lý dữ liệu cho 600 người chơi top đầu ---
id_match1 = []  # Danh sách ID trận đấu của top 600 người chơi
player_lol1 = []  # Danh sách thông tin người chơi

# Lấy danh sách ID trận đấu
for i, puuid_player in enumerate(df_top600):
    try:
        match_ids = get_id_match(puuid_player)
        if match_ids:
            id_match1.extend(match_ids)
        time.sleep(1.2)  # Tránh rate limit của Riot API
    except Exception as e:
        print(f"Lỗi khi lấy match_id từ puuid {puuid_player}: {e}")

# Lấy thông tin người chơi từ ID trận đấu
for i, match_id in enumerate(id_match1):
    try:
        player_info = get_player_from_match(match_id)
        if player_info:
            player_lol1.append(player_info)
        time.sleep(1.2)  # Tránh rate limit
    except Exception as e:
        print(f"Lỗi khi lấy thông tin từ match_id {match_id}: {e}")

# Chuyển dữ liệu thành DataFrame và lưu vào CSV
df_players1 = pd.DataFrame(player_lol1)
df_players1.to_csv("top600_mid_players.csv", index=False)
df_players1.to_csv(r"C:\Users\PC\Desktop\RIOT\top600_mid_players.csv", index=False)

# --- Xử lý dữ liệu cho 1000 người chơi còn lại ---
id_match2 = []  # Danh sách ID trận đấu của 1000 người chơi tiếp theo
player_lol2 = []  # Danh sách thông tin người chơi

# Lấy danh sách ID trận đấu
for i, puuid_player in enumerate(df_remaining):
    try:
        match_ids = get_id_match(puuid_player)
        if match_ids:
            id_match2.extend(match_ids)
        time.sleep(1.2)  # Tránh rate limit
    except Exception as e:
        print(f"Lỗi khi lấy match_id từ puuid {puuid_player}: {e}")

# Lấy thông tin người chơi từ ID trận đấu
for i, match_id in enumerate(id_match2):
    try:
        player_info = get_player_from_match(match_id)
        if player_info:
            player_lol2.append(player_info)
        time.sleep(1.2)  # Tránh rate limit
    except Exception as e:
        print(f"Lỗi khi lấy thông tin từ match_id {match_id}: {e}")

# Chuyển dữ liệu thành DataFrame và lưu vào CSV
df_players2 = pd.DataFrame(player_lol2)
df_players2.to_csv(r"C:\Users\PC\Desktop\RIOT\next1000_mid_players.csv", index=False)
